# Simulating different telescopes
This notebooks provides examples in how to use the lenstronomy.SimulationAPI modules in simulating (realistic) mock lenses taylored to a specific observation and instrument and makes a montage of different telescope settings currently available.

The module enables to use the astronomical magnitude conventions and can translate those into the lenstronomy core module configurations.

In [15]:
import copy
import os
import numpy as np
import scipy
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# make sure lenstronomy is installed, otherwise install the latest pip version

# lenstronomy module import
from lenstronomy.Util import image_util, data_util, util
import lenstronomy.Plots.plot_util as plot_util
from lenstronomy.SimulationAPI.sim_api import SimAPI
from lenstronomy.Plots.plot_util import coordinate_arrows, scale_bar
from astropy.visualization import simple_norm

## Define camera and observations
As an example, we define the camera and observational settings of a LSST-like observation. We define one camera setting and three different observations corresponding to g,r,i imaging.

For the complete list of possible settings, we refer to the SimulationAPI.observation_api classes. There are pre-configured settings which approximately mimic observations from current and future instruments. Be careful using those and check whether they are sufficiently accurate for your specific science case!

In [16]:
# Instrument setting from pre-defined configurations

from lenstronomy.SimulationAPI.ObservationConfig.DES import DES
from lenstronomy.SimulationAPI.ObservationConfig.LSST import LSST
from lenstronomy.SimulationAPI.ObservationConfig.Euclid import Euclid
from lenstronomy.SimulationAPI.ObservationConfig.Roman import Roman

DES_g = DES(band='g', psf_type='GAUSSIAN', coadd_years=3)
DES_r = DES(band='r', psf_type='GAUSSIAN', coadd_years=3)
DES_i = DES(band='i', psf_type='GAUSSIAN', coadd_years=3)
des = [DES_g, DES_r, DES_i]

LSST_g = LSST(band='g', psf_type='GAUSSIAN', coadd_years=10)
LSST_r = LSST(band='r', psf_type='GAUSSIAN', coadd_years=10)
LSST_i = LSST(band='i', psf_type='GAUSSIAN', coadd_years=10)
lsst = [LSST_g, LSST_r, LSST_i]

Roman_g = Roman(band='F062', psf_type='PIXEL', survey_mode='wide_area')
Roman_r = Roman(band='F106', psf_type='PIXEL', survey_mode='wide_area')
Roman_i = Roman(band='F184', psf_type='PIXEL', survey_mode='wide_area')
roman = [Roman_g, Roman_r, Roman_i]

# lenstronomy provides these setting to be imported with the SimulationAPI.ObservationConfig routines.



## Define model settings

The model settings are handled by the SimulationAPI.model_api ModelAPI class. 
The role is to return instances of the lenstronomy LightModel, LensModel, PointSource modules according to the options chosen by the user. Currently, all other model choices are equivalent to the ones provided by LightModel, LensModel, PointSource.
The current options of the class instance only describe a subset of possibilities and we refer to the specific class instances for details about all the possibilities.

For this example, we chose a single lens plane and a single source plane, elliptical Sersic profile for the deflector, the interpolated Galaxy as the source and an additional lensed point source.

In [39]:
kwargs_model = {'lens_model_list': ['EPL', 'SHEAR'],  # list of lens models to be used
                'lens_light_model_list': ['SERSIC_ELLIPSE','SERSIC_ELLIPSE','SERSIC_ELLIPSE'],  # list of unlensed light models to be used
                'source_light_model_list': ['SERSIC_ELLIPSE','SERSIC_ELLIPSE'],  # list of extended source models to be used, here we used the interpolated real galaxy
    }

## Brightness definitions in magnitude space
One core feature is the support of light profile amplitudes in astronomical magnitude space (at least for few selected well defined brightness profiles).

We first define all parameters in magnitude space and then use the SimAPI routine to translate the arguments into lenstronomy conventions used by the ImSim module. The second model of each light component we defined as 'INTERPOL', which sets an interpolation grid given an image. This can be used to past real galaxies as lenses or sources into lenstronomy.

In [40]:
import json
f = open("/global/homes/s/seanjx/gigalens/238/output/238_2024-08-01 16:51:20.124047/asdf.json")
everything = json.load(f)
#everything = {'897':everything['897']}
#del everything['268']

In [41]:
a, b, c = 1, 60, 1000;
t = b*(b-1)/999
print(t)
pee = np.round(np.arange(b)*(np.arange(b)+1)/t)
pee = pee.astype(int)
print(pee)

3.5435435435435436
[  0   1   2   3   6   8  12  16  20  25  31  37  44  51  59  68  77  86
  97 107 119 130 143 156 169 183 198 213 229 246 262 280 298 317 336 356
 376 397 418 440 463 486 510 534 559 584 610 637 664 691 720 748 778 808
 838 869 901 933 966 999]


In [42]:
numpix = 120  # number of pixels per axis of the image to be modelled

# here we define the numerical options used in the ImSim module. 
# Have a look at the ImageNumerics class for detailed descriptions.
# If not further specified, the default settings are used.
kwargs_numerics = {}


def simulate_rgb(ConfigList, size, kwargs_numerics):
    band_b, band_g, band_r = ConfigList
    kwargs_b_band = band_b.kwargs_single_band()
    kwargs_g_band = band_g.kwargs_single_band()
    kwargs_r_band = band_r.kwargs_single_band()
    
    # set number of pixels from pixel scale
    pixel_scale = kwargs_g_band['pixel_scale']
    numpix = int(round(size / pixel_scale))

    sim_b = SimAPI(numpix=numpix, kwargs_single_band=kwargs_b_band, kwargs_model=kwargs_model)
    sim_g = SimAPI(numpix=numpix, kwargs_single_band=kwargs_g_band, kwargs_model=kwargs_model)
    sim_r = SimAPI(numpix=numpix, kwargs_single_band=kwargs_r_band, kwargs_model=kwargs_model)

    # return the ImSim instance. With this class instance, you can compute all the
    # modelling accessible of the core modules. See class documentation and other notebooks.
    imSim_b = sim_b.image_model_class(kwargs_numerics)
    imSim_g = sim_g.image_model_class(kwargs_numerics)
    imSim_r = sim_r.image_model_class(kwargs_numerics)


    # turn magnitude kwargs into lenstronomy kwargs
    kwargs_lens_light_g, kwargs_source_g, kwargs_ps_g = sim_b.magnitude2amplitude(kwargs_lens_light_mag_g, kwargs_source_mag_g)
    kwargs_lens_light_r, kwargs_source_r, kwargs_ps_r = sim_g.magnitude2amplitude(kwargs_lens_light_mag_r, kwargs_source_mag_r)
    kwargs_lens_light_i, kwargs_source_i, kwargs_ps_i = sim_r.magnitude2amplitude(kwargs_lens_light_mag_i, kwargs_source_mag_i)


    image_b = imSim_b.image(kwargs_lens, kwargs_source_g, kwargs_lens_light_g)
    image_g = imSim_g.image(kwargs_lens, kwargs_source_r, kwargs_lens_light_r)
    image_r = imSim_r.image(kwargs_lens, kwargs_source_i, kwargs_lens_light_i)

    # add noise
    image_b += sim_b.noise_for_model(model=image_b)
    image_g += sim_g.noise_for_model(model=image_g)
    image_r += sim_r.noise_for_model(model=image_r)

    # and plot it

    img = np.zeros((image_g.shape[0], image_g.shape[1], 3), dtype=float)
    #scale_max=10000
    def _scale_max(image): 
        flat=image.flatten()
        flat.sort()
        scale_max = flat[int(len(flat)*0.95)]
        return scale_max
    img[:,:,0] = plot_util.sqrt(image_b, scale_min=0, scale_max=_scale_max(image_b))
    img[:,:,1] = plot_util.sqrt(image_g, scale_min=0, scale_max=_scale_max(image_g))
    img[:,:,2] = plot_util.sqrt(image_r, scale_min=0, scale_max=_scale_max(image_r))
    data_class = sim_b.data_class
    return img, data_class

In [49]:
for ii in pee:
    f, axes = plt.subplots(4,4,figsize=(20,20))
    for iii, system in enumerate(everything):

        # g-band
        # lens light
        for iv in everything[system][str(ii)][1]:
            iv['magnitude'] = 10
        for iv in everything[system][str(ii)][2]:
            iv['magnitude'] = 11
        
        kwargs_lens_light_mag_g = everything[system][str(ii)][1]
        # source light
        kwargs_source_mag_g = everything[system][str(ii)][2]


        # and now we define the colors of the other two bands

        # r-band
        g_r_source = 1  # color mag_g - mag_r for source
        g_r_lens = -1  # color mag_g - mag_r for lens light
        kwargs_lens_light_mag_r = copy.deepcopy(kwargs_lens_light_mag_g)
        kwargs_lens_light_mag_r[0]['magnitude'] -= g_r_lens

        kwargs_source_mag_r = copy.deepcopy(kwargs_source_mag_g)
        kwargs_source_mag_r[0]['magnitude'] -= g_r_source



        # i-band
        g_i_source = 2
        g_i_lens = -2
        kwargs_lens_light_mag_i = copy.deepcopy(kwargs_lens_light_mag_g)
        kwargs_lens_light_mag_i[0]['magnitude'] -= g_i_lens

        kwargs_source_mag_i = copy.deepcopy(kwargs_source_mag_g)
        kwargs_source_mag_i[0]['magnitude'] -= g_i_source

        
        kwargs_lens = everything[system][str(ii)][0]
        
        
        
        size = 120*0.065 # width of the image in units of arc seconds

        #img_des, coords_des = simulate_rgb(des, size=size, kwargs_numerics=kwargs_numerics)
        #img_lsst, coords_lss = simulate_rgb(lsst, size=size, kwargs_numerics=kwargs_numerics)
        img_roman, coords_roman = simulate_rgb(roman, size=size, kwargs_numerics=kwargs_numerics)

        ax = np.reshape(axes,16)[iii]
        ax.imshow(img_roman, aspect='equal', origin='lower', extent=[0, size, 0, size])
        ax.set_axis_off()
    plt.subplots_adjust(wspace=0.1)
    plt.savefig("animation/"+f"{ii: 07d}.png",transparent=True)
    plt.close()

In [32]:
print(np.max(img_roman))

1.0
